# Part-of-Speech Tagging for Portuguese

<div class="admonition note" name="html-admonition" style="background: lightblue; padding: 10px">
<p class="title">Note</p>
This section, "Working in Languages Beyond English," is co-authored with <a href="http://www.quinndombrowski.com/">Quinn Dombrowski</a>, the Academic Technology Specialist at Stanford University and a leading voice in multilingual digital humanities. I'm grateful to Quinn for helping expand this textbook to serve languages beyond English. 
</div>

In this lesson, we're going to learn about the textual analysis methods *part-of-speech tagging* and *keyword extraction* for Portuguese texts. These methods will help us computationally parse sentences and better understand words in context.

---

## spaCy and Natural Language Processing (NLP)

To computationally identify parts of speech, we're going to use the natural language processing library spaCy. For a more extensive introduction to NLP and spaCy, see the previous lesson.

To parse sentences, spaCy relies on machine learning models that were trained on large amounts of labeled text data. If you've used the preprocessing or named entity recognition notebooks for this language, you can skip the steps for installing spaCy and downloading the language model.

## Install spaCy

To use spaCy, we first need to install the library.

In [ ]:
!pip install -U spacy

## Import Libraries

Then we're going to import `spacy` and `displacy`, a special spaCy module for visualization.

In [1]:
import spacy
from spacy import displacy
from collections import Counter
import pandas as pd
pd.options.display.max_rows = 400
pd.options.display.max_colwidth = 400

We're also going to import the `Counter` module for counting nouns, verbs, adjectives, etc., and the `pandas` library for organizing and displaying data (we're also changing the pandas default max row and column width display setting).

## Download Language Model

Next we need to download the Portuguese-language model (`pt_core_news_md`), which will be processing and making predictions about our texts. This is the model that was trained on both European (CETEMPúblico) and Brazilian (CETENFolha) variants. You can download the `pt_core_news_md` model by running the cell below:

In [2]:
!python -m spacy download pt_core_news_md

     ---------------------------------------- 0.0/42.4 MB ? eta -:--:--
     ---------------------------------------- 0.2/42.4 MB 6.9 MB/s eta 0:00:07
      --------------------------------------- 0.8/42.4 MB 10.1 MB/s eta 0:00:05
     - -------------------------------------- 1.3/42.4 MB 10.0 MB/s eta 0:00:05
     - -------------------------------------- 1.8/42.4 MB 10.1 MB/s eta 0:00:05
     -- ------------------------------------- 2.1/42.4 MB 9.8 MB/s eta 0:00:05
     -- ------------------------------------- 2.5/42.4 MB 9.5 MB/s eta 0:00:05
     -- ------------------------------------- 3.0/42.4 MB 9.5 MB/s eta 0:00:05
     --- ------------------------------------ 3.4/42.4 MB 9.5 MB/s eta 0:00:05
     --- ------------------------------------ 3.8/42.4 MB 9.3 MB/s eta 0:00:05
     ---- ----------------------------------- 4.3/42.4 MB 9.4 MB/s eta 0:00:05
     ---- ----------------------------------- 4.8/42.4 MB 9.5 MB/s eta 0:00:04
     ---- ----------------------------------- 5.2/42.4 M

*Note: spaCy offers [models for other languages](https://spacy.io/usage/models#languages) including German, French, Spanish, Portuguese, Russian, Italian, Dutch, Greek, Norwegian, and Lithuanian*.  

*spaCy offers language and tokenization support for other language via external dependencies — such as [PyviKonlpy](https://github.com/konlpy/konlpy) for Korean.*

## Load Language Model

Once the model is downloaded, we need to load it with `spacy.load()` and assign it to the variable `nlp`.

In [3]:
nlp = spacy.load('pt_core_news_md')

## Create a Processed spaCy Document

Whenever we use spaCy, our first step will be to create a processed spaCy `document` with the loaded NLP model `nlp()`. Most of the heavy NLP lifting is done in this line of code. After processing, the `document` object will contain tons of juicy language data — named entities, sentence boundaries, parts of speech — and the rest of our work will be devoted to accessing this information.

In [4]:
filepath = '../texts/pt.txt'
text = open(filepath, encoding='utf-8').read()
document = nlp(text)

## spaCy Part-of-Speech Tagging
The tags that spaCy uses for part-of-speech are based on work done by [Universal Dependencies](https://universaldependencies.org/), an effort to create a set of part-of-speech tags that work across many different languages. Texts from various languages are annotated using this common set of tags, and contributed to a common repository that can be used to train models like spaCy.

The Universal Dependencies page has information about the annotated corpora available for each language; it's worth looking into the corpora that were annotated for your language.

| POS   | Description               | Examples                                      |
|:-----:|:-------------------------:|:---------------------------------------------:|
| ADJ   | adjective                 | big, old, green, incomprehensible, first      |
| ADP   | adposition                | in, to, during                                |
| ADV   | adverb                    | very, tomorrow, down, where, there            |
| AUX   | auxiliary                 | is, has (done), will (do), should (do)        |
| CONJ  | conjunction               | and, or, but                                  |
| CCONJ | coordinating conjunction  | and, or, but                                  |
| DET   | determiner                | a, an, the                                    |
| INTJ  | interjection              | psst, ouch, bravo, hello                      |
| NOUN  | noun                      | girl, cat, tree, air, beauty                  |
| NUM   | numeral                   | 1, 2017, one, seventy-seven, IV, MMXIV        |
| PART  | particle                  | ’s, not,                                      |
| PRON  | pronoun                   | I, you, he, she, myself, themselves, somebody |
| PROPN | proper noun               | Mary, John, London, NATO, HBO                 |
| PUNCT | punctuation               | ., (, ), ?                                    |
| SCONJ | subordinating conjunction | if, while, that                               |
| SYM   | symbol                    | $, %, §, ©, +, −, ×, ÷, =, :), 😝             |
| VERB  | verb                      | run, runs, running, eat, ate, eating          |
| X     | other                     | sfpksdpsxmsa                                  |
| SPACE | space                     |                                               |


Above is a POS chart taken from [spaCy's website](https://spacy.io/api/annotation#named-entities), which shows the different parts of speech that spaCy can identify as well as their corresponding labels. To quickly see spaCy's POS tagging in action, we can use the [spaCy module `displacy`](https://spacy.io/usage/visualizers#ent) on our sample `document` with the `style=` parameter set to "dep" (short for dependency parsing):

## Get Part-Of-Speech Tags

To get part of speech tags for every word in a document, we have to iterate through all the tokens in the document and pull out the `.lemma_` attribute for each token, which gives us the un-inflected version of the word. We'll also pull out the  `.pos_` attribute for each token. We can get even finer-grained dependency information with the attribute `.dep_`.


In [5]:
for token in document:
    print(token.lemma_, token.pos_, token.dep_)

PRIMEIRA NUM ROOT
PARTE NOUN case





 SPACE dep
UMA NUM ROOT
HISTORIA NOUN nmod
VERDADEIRA PROPN ROOT



 SPACE dep
I PROPN amod


 SPACE dep
ser AUX cop
um DET det
physionomia NOUN ROOT
incaracteristico ADJ amod
, PUNCT punct
apagar VERB acl
, PUNCT punct
tristissimo ADJ conj
. PUNCT punct


 SPACE dep
não ADV advmod
se PRON expl
poder VERB ROOT
dizer VERB xcomp
o DET det
idade NOUN obj
que PRON nsubj
ter VERB acl:relcl
, PUNCT punct
nem CCONJ cc
mesmo ADV fixed
se PRON expl
ter VERB conj
idade NOUN obj
. PUNCT punct


 SPACE dep
tanto ADV advmod
poder VERB ROOT
ter VERB xcomp
trinta NUM obj
ou CCONJ cc
quarenta NUM conj
como ADP case
setenta NUM nummod
annos PROPN obl
. PUNCT punct


 SPACE dep
curvar VERB ROOT
por o ADP case
idade NOUN obl:agent
ou CCONJ cc
por o ADP case
desgosto NOUN conj
? PUNCT punct
Encanecido ADV ROOT
porque SCONJ mark
o DET det
anno NOUN nsubj
ter VERB advcl

 SPACE dep
corrido ADJ xcomp
por ADP case
sobre ADP case
o DET det
cabeça NOUN obl
de 'elle PRON ap

## Practicing with the example text
When working with languages that have inflection, we typically use `token.lemma_` instead of `token.text` like you'll find in the English examples. This is important when we're counting, so that differently-inflected forms of a word (e.g. masculine vs. feminine or singular vs. plural) aren't counted as if they were different words.

In [6]:
filepath = "../texts/pt.txt"
document = nlp(open(filepath, encoding="utf-8").read())

## Get Adjectives

| POS   | Description               | Examples                                      |
|:-----:|:-------------------------:|:---------------------------------------------:|
| ADJ   | adjective                 | big, old, green, incomprehensible, first      |

To extract and count the adjectives in the example text, we will follow the same model as above, except we'll add an `if` statement that will pull out words only if their POS label matches "ADJ."

<div class="admonition pythonreview" name="html-admonition" style="background: lightgreen; padding: 10px">
<p class="title">Python Review</p>

While we demonstrate how to extract parts of speech in the sections below, we're also going to reinforce some integral Python skills. Notice how we use `for` loops and `if` statements to `.append()` specific words to a list. Then we count the words in the list and make a pandas dataframe from the list.
</div>

Here we make a list of the adjectives identified in the example text:

In [7]:
adjs = []
for token in document:
    if token.pos_ == 'ADJ':
        adjs.append(token.lemma_)

In [8]:
adjs

['incaracteristico',
 'tristissimo',
 'corrido',
 'debei',
 'rachitico',
 'incompleto',
 'gorgeada',
 'escarlate',
 'estranho',
 'doloroso',
 'primeiro',
 'fraco',
 'inerme',
 'pobre',
 'dependente',
 'aquelle',
 'rude',
 'moral',
 'duro',
 'opulento',
 'voluntario',
 'inconsciente',
 'proprio',
 'mesmo',
 'inquieto',
 'medroso',
 'primeiro',
 'alto',
 'plethorico',
 'grosso',
 'farto',
 'suzerano',
 'ella',
 'bonito',
 'flexivel',
 'branco',
 'ideal',
 'inglez',
 'gracioso',
 'delicado',
 'frio',
 'metalico',
 'fino',
 'esguio',
 'branco',
 'fraquinho',
 'grotesco',
 'informe',
 'inteiro',
 'inteiro',
 'intenso',
 'dilacerante',
 'estranho',
 'altivo',
 'fria',
 'desdenhoso',
 'repugnante',
 'imperturbavel',
 'olympico',
 'glacial',
 'original',
 'accentuar',
 'ironico',
 'irregular',
 'obscuro',
 'infeliz',
 'velho',
 'grotesco',
 'aspera',
 'hostil',
 'extravagante',
 'pequeno',
 'magro',
 'comico',
 'bom',
 'unico',
 'singular',
 'faiscante',
 'sonoro',
 'rico',
 'estranho',
 'máo'

Then we count the unique adjectives in this list with the `Counter()` module:

In [9]:
adjs_tally = Counter(adjs)

In [10]:
adjs_tally.most_common()

[('grande', 98),
 ('bom', 79),
 ('pequeno', 58),
 ('pobre', 47),
 ('alto', 44),
 ('velho', 43),
 ('primeiro', 38),
 ('mesmo', 35),
 ('cheio', 35),
 ('ella', 34),
 ('pequenino', 33),
 ('rico', 30),
 ('doce', 30),
 ('feliz', 30),
 ('branco', 29),
 ('querido', 29),
 ('proprio', 28),
 ('louro', 26),
 ('novo', 24),
 ('unico', 23),
 ('bonito', 21),
 ('elle', 21),
 ('moderno', 21),
 ('longo', 20),
 ('elegante', 20),
 ('doloroso', 19),
 ('fino', 18),
 ('adoravel', 18),
 ('verdadeiro', 18),
 ('forte', 18),
 ('estranho', 17),
 ('brilhante', 17),
 ('anno', 17),
 ('profundo', 17),
 ('altivo', 16),
 ('alegre', 16),
 ('azul', 16),
 ('intimo', 15),
 ('mysterioso', 15),
 ('gracioso', 14),
 ('antigo', 14),
 ('digno', 13),
 ('vivo', 13),
 ('capaz', 13),
 ('formoso', 13),
 ('simples', 13),
 ('preciso', 13),
 ('triste', 13),
 ('delicado', 12),
 ('inteiro', 12),
 ('obscuro', 12),
 ('luminoso', 12),
 ('nervoso', 12),
 ('humilde', 12),
 ('irresistivel', 12),
 ('raro', 12),
 ('puro', 12),
 ('opulento', 11),
 

Then we make a dataframe from this list:

In [11]:
df = pd.DataFrame(adjs_tally.most_common(), columns=['adj', 'count'])
df[:100]

,adj,count
0,grande,98
1,bom,79
2,pequeno,58
3,pobre,47
4,alto,44
5,velho,43
6,primeiro,38
7,mesmo,35
8,cheio,35
9,ella,34


## Get Nouns

| POS   | Description               | Examples                                      |
|:-----:|:-------------------------:|:---------------------------------------------:|
| NOUN  | noun                      | girl, cat, tree, air, beauty                  |

To extract and count nouns, we can follow the same model as above, except we will change our `if` statement to check for POS labels that match "NOUN".

In [12]:
nouns = []
for token in document:
    if token.pos_ == 'NOUN':
        nouns.append(token.lemma_)

nouns_tally = Counter(nouns)

df = pd.DataFrame(nouns_tally.most_common(), columns=['noun', 'count'])
df[:100]

,noun,count
0,_,417
1,vida,131
2,dia,127
3,vez,103
4,mulher,103
5,homem,100
6,casa,97
7,olho,93
8,cousa,74
9,filha,70


## Get Verbs

| POS   | Description               | Examples                                      |
|:-----:|:-------------------------:|:---------------------------------------------:|
| VERB  | verb                      | run, runs, running, eat, ate, eating          |

To extract and count works of art, we can follow a similar-ish model to the examples above. This time, however, we're going to make our code even more economical and efficient (while still changing our `if` statement to match the POS label "VERB").

<div class="admonition pythonreview" name="html-admonition" style="background: lightgreen; padding: 10px">
<p class="title">Python Review</p>

We can use a [*list comprehension*](https://melaniewalsh.github.io/Intro-Cultural-Analytics/Python/More-Lists-Loops.html#List-Comprehensions) to get our list of verbs in a single line of code! Closely examine the first line of code below:
</div>

In [13]:
verbs = [token.lemma_ for token in document if token.pos_ == 'VERB']

verbs_tally = Counter(verbs)

df = pd.DataFrame(verbs_tally.most_common(), columns=['verb', 'count'])
df[:100]

,verb,count
0,ter,334
1,fazer,167
2,saber,133
3,dizer,124
4,dar,100
5,haver,89
6,ver,89
7,ser,80
8,poder,74
9,querer,68


# Keyword Extraction

## Get Sentences with Keyword

spaCy can also identify sentences in a document. To access sentences, we can iterate through `document.sents` and pull out the `.text` of each sentence.

We can use spaCy's sentence-parsing capabilities to extract sentences that contain particular keywords, such as in the function below. Note that the function assumes that the keyword provided will be exactly the same as it appears in the text (e.g. matching all inflection for case, number, gender, etc. As a Spanish example, if you use `bueno` as the keyboard, it won't match `buena` or `buenos`.)

With the function `find_sentences_with_keyword()`, we will iterate through `document.sents` and pull out any sentence that contains a particular "keyword." Then we will display these sentence with the keywords bolded.

In [14]:
import re
from IPython.display import Markdown, display

In [15]:
def find_sentences_with_keyword(keyword, document):
    
    #Iterate through all the sentences in the document and pull out the text of each sentence
    for sentence in document.sents:
        sentence = sentence.text
        
        #Check to see if the keyword is in the sentence (and ignore capitalization by making both lowercase)
        if keyword.lower() in sentence.lower():
            
            #Use the regex library to replace linebreaks and to make the keyword bolded, again ignoring capitalization
            sentence = re.sub('\n', ' ', sentence)
            sentence = re.sub(f"{keyword}", f"**{keyword}**", sentence, flags=re.IGNORECASE)
            
            display(Markdown(sentence))

In [16]:
find_sentences_with_keyword(keyword="bom", document=document)

Todos o conheciam, todos o repetiam em alto e **bom** som para que elle o não ignorasse, mas _ella_ amava-o; ella não o dizia, não o pensava, não o tinha notado sequer!  

Tudo que houvera **bom** na sua vida lhe tinha vindo d'ella.  

Surprehendia-se ás vezes doentiamente, a fazer planos impossiveis antes de adormecer; a imaginar quanto seria **bom** ser muito rico, viver na alta roda, n'aquella esphera aristocratica e distincta em que se não trabalha, em que se falla de um modo especial e caracteristico, com termos escolhidos, com inflexões muito mais suaves, com uns certos desdens que d'antes lhe pareciam ridiculos e que lhe estavam agora parecendo superiormente requintados.

Á entrada dos dous a mãe teve um comprimento um pouco secco, a filha um sorriso de graça adoravel, de garridice innata mas irresistivel.  --Quiz vêl-o porque soube que tem sido muito **bom** para Thadeu, excellente mesmo.

Sempre é **bom**.

...

É **bom** que tenhamos isto sempre bem presente, para que não sejamos accintosamente inimigos do que foi, nem loucamente vaidosos do que vai ser.  

É **bom** que o repitamos: ha dezenove seculos que a humanidade caminha, sem parar um só instante.  Tem tido dias que podem chamar-se seculos, e em compensação tem tido seculos que pódem chamar-se dias; em todo o caso, é porque ella ainda não estacionou que hoje avista emfim o ponto a que se dirigia.  Tenhamos o santo orgulho do que temos feito, mas não desprezemos o que os outros fizeram antes de nós.         

Nunca se deu ao trabalho de improvisar arrojos de eloquencia, tinha sempre ao serviço das suas convicções umas anecdotas a um tempo cheias de graça e de **bom** senso, umas pequenas historias que deitavam por terra toda a laboriosa rethorica dos adversarios.  

Não que elle fizesse nenhum d'esses discursos tribunicios que arrastam e enthusiasmam as massas, mas sempre pela força irresistivel do seu **bom** senso, que era tão raro e tão completo, que o punha a par dos homens de genio.  

Marion, de Magdalena impudica e triumphante, levanta-se Magdalena arrependida e piedosa, e Esmeralda não tem a esmola, a caridade de um sorriso **bom** para Quasimodo!  

## Get Keyword in Context

We can also find out about a keyword's more immediate context — its neighboring words to the left and right — and we can fine-tune our search with POS tagging.

To do so, we will first create a list of what's called *ngrams*. "Ngrams" are any sequence of *n* tokens in a text. They're an important concept in computational linguistics and NLP. (Have you ever played with [Google's *Ngram* Viewer](https://books.google.com/ngrams)?)

Below we're going to make a list of *bigrams*, that is, all the two-word combinations from the sample text. We're going to use these bigrams to find the neighboring words that appear alongside particular keywords.

In [17]:
#Make a list of tokens and POS labels from document if the token is a word 
tokens_and_labels = [(token.text, token.pos_) for token in document if token.is_alpha]

In [18]:
#Make a function to get all two-word combinations
def get_bigrams(word_list, number_consecutive_words=2):
    
    ngrams = []
    adj_length_of_word_list = len(word_list) - (number_consecutive_words - 1)
    
    #Loop through numbers from 0 to the (slightly adjusted) length of your word list
    for word_index in range(adj_length_of_word_list):
        
        #Index the list at each number, grabbing the word at that number index as well as N number of words after it
        ngram = word_list[word_index : word_index + number_consecutive_words]
        
        #Append this word combo to the master list "ngrams"
        ngrams.append(ngram)
        
    return ngrams

In [19]:
bigrams = get_bigrams(tokens_and_labels)

Let's take a peek at the bigrams:

In [20]:
bigrams[5:20]

[[('Era', 'AUX'), ('uma', 'DET')],
 [('uma', 'DET'), ('physionomia', 'NOUN')],
 [('physionomia', 'NOUN'), ('incaracteristica', 'ADJ')],
 [('incaracteristica', 'ADJ'), ('apagada', 'VERB')],
 [('apagada', 'VERB'), ('tristissima', 'ADJ')],
 [('tristissima', 'ADJ'), ('Não', 'ADV')],
 [('Não', 'ADV'), ('se', 'PRON')],
 [('se', 'PRON'), ('podia', 'VERB')],
 [('podia', 'VERB'), ('dizer', 'VERB')],
 [('dizer', 'VERB'), ('a', 'DET')],
 [('a', 'DET'), ('idade', 'NOUN')],
 [('idade', 'NOUN'), ('que', 'PRON')],
 [('que', 'PRON'), ('tinha', 'VERB')],
 [('tinha', 'VERB'), ('nem', 'CCONJ')],
 [('nem', 'CCONJ'), ('mesmo', 'ADV')]]

Now that we have our list of bigrams, we're going to make a function `get_neighbor_words()`. This function will return the most frequent words that appear next to a particular keyword. The function can also be fine-tuned to return neighbor words that match a certain part of speech by changing the `pos_label` parameter.

In [21]:
def get_neighbor_words(keyword, bigrams, pos_label = None):
    
    neighbor_words = []
    keyword = keyword.lower()
    
    for bigram in bigrams:
        
        #Extract just the lowercased words (not the labels) for each bigram
        words = [word.lower() for word, label in bigram]        
        
        #Check to see if keyword is in the bigram
        if keyword in words:
            
            for word, label in bigram:
                
                #Now focus on the neighbor word, not the keyword
                if word.lower() != keyword:
                    #If the neighbor word matches the right pos_label, append it to the master list
                    if label == pos_label or pos_label == None:
                        neighbor_words.append(word.lower())
    
    return Counter(neighbor_words).most_common()

In [22]:
get_neighbor_words("bom", bigrams)

[('é', 10),
 ('que', 8),
 ('e', 7),
 ('do', 6),
 ('um', 6),
 ('de', 4),
 ('o', 3),
 ('na', 2),
 ('muito', 2),
 ('para', 2),
 ('nadador', 2),
 ('sorriso', 2),
 ('deus', 2),
 ('senso', 2),
 ('som', 1),
 ('houvera', 1),
 ('seria', 1),
 ('ser', 1),
 ('agora', 1),
 ('eu', 1),
 ('cão', 1),
 ('sempre', 1),
 ('sério', 1),
 ('vae', 1),
 ('era', 1),
 ('das', 1),
 ('homem', 1),
 ('nome', 1),
 ('preço', 1),
 ('a', 1),
 ('ter', 1),
 ('estava', 1),
 ('disse', 1),
 ('capitão', 1),
 ('sahiu', 1),
 ('vaes', 1),
 ('tão', 1),
 ('minha', 1),
 ('jantar', 1),
 ('humor', 1),
 ('cheiro', 1),
 ('ir', 1),
 ('ou', 1),
 ('mas', 1),
 ('sympathico', 1),
 ('com', 1),
 ('intimo', 1),
 ('quasi', 1),
 ('será', 1),
 ('seu', 1)]

In [23]:
get_neighbor_words("bom", bigrams, pos_label='VERB')

[('houvera', 1), ('ser', 1), ('ter', 1), ('disse', 1), ('ir', 1)]

## Your Turn!

Try out `find_sentences_with_keyword()` and `get_neighbor_words` with your own keywords of interest.

In [24]:
find_sentences_with_keyword(keyword="que", document=document)

Não se podia dizer a idade **que** tinha, nem mesmo se tinha idade.  

Encanecido por**que** os annos tinham corrido por sobre a cabeça d'elle, ou por**que** lhe tinham pesado duplamente sobre os hombros debeis?  

**que**m o podia dizer?  

Para elle com certeza **que** a adolescencia não tivera as suas madrugadas azues tão gorgeadas e tão festivas, nem a virilidade tivera a fanfarra estridente dos seus clarins, a florescencia escarlate dos seus rosaes voluptuosos.  

Dependêra de todos primeiro por**que** era fraco e inerme, depois por**que** fôra pobre, dependente, sem a**que**lla aspera dignidade **que** os atrictos da vida tornam mais rude e **que** é a armadura moral **que** salvaguarda o homem nos duros combates sociaes.  

...

Dir-nos-hão os poetas de hoje **que** ellas não existem, e, o **que** é peior, **que** ellas não puderam existir nunca.  

é bem triste, é bem esteril a arte **que** só trata de rebaixar o **que** em nós é de mais elevada essencia, e só **que**r **que** vejamos a fatalidade brutal do instincto, onde viamos d'antes a fatalidade mais nobre do sentimento.  

Não acreditemos o **que** elles nos dizem, por**que** na sua preoccupação exagerada do horrivel, elles mentem muito mais do **que** os outros mentiam na sua preoccupação exagerada do bello!  

Estes reunindo todos os vicios e hediondezas **que** encontraram dispersos n'uma só figura, conseguem apenas crear... um monstro, um ser hybrido e infecundo **que** a ninguem aproveita!  

Os outros synthetisando n'uma filha do seu genio as harmonias, as feições, os encantos, **que** estudaram e amaram em toda a natureza, conseguiram alguma coisa mais!  

In [25]:
get_neighbor_words("que", bigrams, pos_label=None)

[('o', 231),
 ('a', 176),
 ('se', 109),
 ('em', 106),
 ('é', 103),
 ('não', 83),
 ('e', 81),
 ('elle', 78),
 ('lhe', 66),
 ('de', 63),
 ('ella', 57),
 ('os', 54),
 ('eu', 45),
 ('do', 45),
 ('era', 41),
 ('tudo', 35),
 ('as', 33),
 ('com', 33),
 ('me', 29),
 ('para', 26),
 ('como', 26),
 ('tinha', 25),
 ('mais', 23),
 ('nunca', 21),
 ('um', 21),
 ('por', 19),
 ('mas', 18),
 ('ha', 17),
 ('elles', 17),
 ('uma', 16),
 ('só', 15),
 ('havia', 15),
 ('dos', 14),
 ('na', 14),
 ('homem', 14),
 ('foi', 14),
 ('todos', 14),
 ('nos', 14),
 ('tem', 13),
 ('mulher', 13),
 ('margarida', 12),
 ('ao', 11),
 ('já', 11),
 ('ainda', 11),
 ('no', 11),
 ('sabia', 10),
 ('gente', 10),
 ('ninguem', 10),
 ('thadeu', 9),
 ('visto', 9),
 ('são', 9),
 ('pessoas', 9),
 ('diz', 9),
 ('aquella', 9),
 ('ellas', 9),
 ('tanto', 8),
 ('cousa', 8),
 ('tambem', 8),
 ('pae', 8),
 ('ás', 8),
 ('verdade', 8),
 ('fôra', 8),
 ('creio', 8),
 ('bom', 8),
 ('alegrias', 7),
 ('depois', 7),
 ('então', 7),
 ('soube', 7),
 ('lhes',